In [ ]:
from FNN import FNN
from Layer import Layer
from Neuron import Neuron
import numpy as np
from ActivFunctions import  *
from InitFunctions import  *
from SuppFunctions import  *
from ErrorClasses import *

Neuron Class functionality testing

In [ ]:
inputTest = np.array([1.0,1.0])

In [ ]:
#zero initialization
t0NeuronZero = Neuron(2,identity)
print(t0NeuronZero.weight_vector.shape)
t0NeuronZero.forward(inputTest)

In [ ]:
#random initialization
t0NeuronRandom = Neuron(2,identity,method_ini="Random")
print(t0NeuronRandom.weight_vector.shape)
t0NeuronRandom.forward(inputTest)

In [ ]:
#initialization from list
vector = [1.0,2.0,3.5]
print(vector)
tNeuron = Neuron(vector,identity)
print(tNeuron.weight_vector.shape)
tNeuron.forward(inputTest)

In [ ]:
#initialization from vector
vector = np.array([1.0,2.0,3.5])
print(vector)
print(vector.shape)
t1Neuron = Neuron(vector,identity)
print(t1Neuron.weight_vector.shape)
t1Neuron.forward(inputTest)

In [ ]:
#initialization from row vector(2D array)
vector_row = vector[np.newaxis,:]
print(vector_row)
print(vector_row.shape)
t2Neuron = Neuron(vector_row,identity)
print(t2Neuron.weight_vector.shape)
t2Neuron.forward(inputTest)

In [ ]:
#initialization from column vector(2D array)
vector_column = vector[:,np.newaxis]
print(vector_column)
print(vector_column.shape)
t3Neuron = Neuron(vector_column,identity)
print(t3Neuron.weight_vector.shape)
t3Neuron.forward(inputTest)

In [ ]:
#array input
t0NeuronRandom = Neuron(2,identity,method_ini="Random")
inputTest = np.array([[1.0,1.0],[1.0,1.0]])
print(inputTest.shape)
print(inputTest)
print(t0NeuronRandom.weight_vector.shape)
t0NeuronRandom.forward(inputTest)

In [ ]:
#list input
vector = np.array([1.0,2.0,3.5])
t0NeuronRandom = Neuron(vector,identity,method_ini="Random")
inputTest = ["1.0","1.0"]
t0NeuronRandom.forward(inputTest)

In [ ]:
#error messages testing
print("Test for: no function given as activation function")
try:
    vector = np.array([1.0,2.0,3.5])
    errorNeuron = Neuron(vector,1)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (2 dim)")
try:
    vector = np.array([[1.0,2.0,3.5],[1.0,2.0,3.5]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not vector given for weights initialization (higher than 2 dim)")
try:
    vector = np.array([[[1.0,2.0,3.5],[1.0,2.0,3.5]],[[1.0,2.0,3.5],[1.0,2.0,3.5]]])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong weights given for initialization")
try:
    vector = np.array(["1.0","2.0","3.5"])
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: wrong var type given in list for weight initialization")
try:
    vector = ["as","2.0","3.5"]
    errorNeuron = Neuron(vector,identity)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - inproper dimension of input")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="Random")
    inputTest = np.array([[[1.0,1.0],[1.0,1.0]],[[1.0,1.0],[1.0,1.0]]]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - wrong value types in input: they are not rational numbers")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="Random")
    inputTest = np.array([1.0 + 3j,1.0]) 
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type for list")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="Random")
    inputTest = ["a","1.0"]
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given variable is of not supported type")
try:
    vector = np.array([1.0,2.0,3.5])
    t0NeuronRandom = Neuron(vector,identity,method_ini="Random")
    inputTest = "a"
    t0NeuronRandom.forward(inputTest)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too small)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorSmall = np.array([1.0])
    neuronTest.forward(inputTestErrorSmall)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: input propagation (forward pass) - given input is too small to match neuron dimensions")
try:
    #wrong input (too big)
    vector = np.array([1.0,2.0,3.5])
    neuronTest = Neuron(vector,identity)
    inputTestErrorBig = np.array([[1.0],[1.0],[1.0],[1.0]])
    neuronTest.forward(inputTestErrorBig)
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error:",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")

Layer class implementation testing

In [ ]:
#initialization by number of dim in vector
dim_layer = np.array([4,5])
testLayer = Layer(dim_layer,identity,method_ini="Random")
testLayer.weights_array
testLayer.activ_functions

In [ ]:
#forward test pass
inputTestVect = np.array([[1.0],[1.0],[1.0],[1.0],[1.0]])
inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])

print(testLayer.forward(inputTestVect))
print(testLayer.forward(inputTestArray))

FNN class implementation testing

In [ ]:
#initialization by number of dim in vector
dim_layer = np.array([5,5,3,2,1])
testNet = FNN(dim_layer,identity,method_ini="Random")
print(testNet.weights_list[0])
print(testNet.weights_list[1])
print(testNet.weights_list[2])
print(testNet.weights_list[3])
print(testNet.activ_functions_list[0])
print(testNet.activ_functions_list[1])
print(testNet.activ_functions_list[2])
print(testNet.activ_functions_list[3])
#forward test pass
inputTestVect = np.array([[1.0],[1.0],[1.0],[1.0],[1.0]])
inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])

print(testNet.forward(inputTestVect))
print(testNet.forward(inputTestArray))


In [ ]:
#error messages testing
print("Test for: not supported input for weights")
try:
    dim_layer = "[5,5,3,2,1]"
    testNet = FNN(dim_layer,identity,method_ini="Random")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: array of not supported dimension")
try:
    dim_layer = np.array([[5,5,3,2,1],[5,5,3,2,1]])
    testNet = FNN(dim_layer,identity,method_ini="Random")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedArrayDimGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: given numbers are not integers")
try:
    dim_layer = np.array([5,5,3.1,2,1])
    testNet = FNN(dim_layer,identity,method_ini="Random")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")
#
print("Test for: not enough numbers given. Minimally required input and output layers cannot be created")
try:
    dim_layer = np.array([5])
    testNet = FNN(dim_layer,identity,method_ini="Random")
except Exception as error_caught:
    if(isinstance(error_caught, NotSupportedInputGiven)):
        print("Correct errors was caught. Error: ",error_caught)
    else:
       print("Something went wrong: wrong error caught. Error: ", error_caught) 
else:
    print("Something went wrong: no error caught")